In [9]:
#| default_exp sharedUtilities
#| default_cls_lvl 3

# Shared utilties

In [3]:
#| export
import math
import xarray as xr
from glob import glob
from tqdm import tqdm 
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchmetrics import Metric
import matplotlib.pyplot as plt

In [2]:
#| export
def preprocess(ds):
    """
    Used data has overlapping time data since they contain one month before the actual year
    and one month after the actual year.
    The function selects only the year of interest without the overlapping time data.

    Args:
        ds (xr.Dataset): netCDF4 data

    Returns:
        xr.Dataset: data without time overlap
    """
    return ds.sel(time=str(ds.time[len(ds.time)//2].dt.year.data)).resample(time="1D").mean()


In [3]:
#| export
def read_netcdfs(files, dim, transform_func, transform_calendar=None, cftime = True):
    """Reads multiples netcdfs files"""
    def process_one_path(path):
        try:
            if transform_calendar is not None:
                calendar = False
            else:
                calendar = True
            with xr.open_dataset(path, decode_times = calendar, use_cftime = cftime) as ds:
                if transform_calendar is not None:
                    ds[dim].attrs['calendar'] = transform_calendar
                    ds = xr.decode_cf(ds, use_cftime = cftime)
                if transform_func is not None:
                    ds = transform_func(ds)
                ds.load()
                return ds
        except Exception as e:
            print(path)
            print(e)
            return
    paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in tqdm(paths)]
    datasets = [x for x in datasets if x is not None]
    combined = xr.concat(datasets, dim)
    return combined

In [4]:
#| export
def transform_calendar(ds,
                       timedim="time",
                       calendarname="proleptic_gregorin"):
    """Transforms calendar of time index in xarray dataset"""
    ds[timedim].attrs['calendar'] = calendarname
    return ds

In [ ]:
#| export
class TaylorDiagram(object):
    """
    Taylor diagram.
    Plot model standard deviation and correlation to reference (data)
    sample in a single-quadrant polar plot, with r=stddev and
    theta=arccos(correlation).
    """
    def __init__(self, refdata, ax,  *args, **kwargs):
        # memorize reference data
        self.refdata = refdata
        self.ax = ax
        refstd = refdata.std(ddof=1) 
        rlim = [0.0, 1.2*refstd]
        thetalim = [0.0, 0.5*np.pi]
        if ax != None:
            # Add reference point and stddev contour
            try:
                kwargs["marker"]
            except:
                kwargs["marker"] = 'o'
            try:
                kwargs["color"]
            except:
                kwargs["color"] = "black"
            try:
                kwargs["ms"]
            except:
                kwargs["ms"] = 10     
            try:
                kwargs["label"]
            except:
                kwargs["label"] = "reference"                     
            ax.plot([0], refstd, *args, ls='', zorder=0, **kwargs)
            ax.grid(True, linestyle="--")
            
            ax.set_xlabel("St. dev.")
            ax.xaxis.set_label_coords(0.5, -0.1)
            ax.set_xlim(thetalim)
            ax.set_ylim(rlim)
            self.rlim = rlim
            self.thetalim = thetalim
        self.model_std = {"reference" : refstd}
        self.corrcoef = {"reference" : 1.0}
        self.rms = {"reference" : 0.0}
    def add_sample(self, model_data, *args, **kwargs):
        """
        Add sample to the Taylor
        diagram. *args* and *kwargs* are directly propagated to the
        `Figure.plot` command.
        """
        model_std = model_data.std(ddof=1)
        corrcoef = np.corrcoef(model_data, self.refdata)[0, 1]
        rms = np.sqrt(self.model_std["reference"]**2 + model_std**2 - 2.0*self.model_std["reference"]*model_std*np.cos(np.arccos(corrcoef)))
        try:
            label = kwargs["label"]
        except:
            label = "model"+str(len(self.model_std.keys())) 
        if self.ax != None:
            self.ax.plot(np.arccos(corrcoef), model_std,
                            *args, **kwargs)  # (theta, radius)
            if model_std > self.rlim[1]:
                self.rlim = [0, 1.2 * model_std]
                self.ax.set_ylim(*self.rlim)
            if np.arccos(corrcoef) > self.thetalim[1]:
                self.thetalim[1] = 1.2 * np.arccos(corrcoef)
                self.ax.set_xlim(*self.thetalim)                       
        self.model_std[label] = model_std
        self.corrcoef[label] = corrcoef
        self.rms[label] = rms
        #self.ax.set_ylim(rlim) 
    def get_samples(self):
        return self.model_std, self.corrcoef, self.rms
    def finalize(self):
        
        if self.ax is None:
            return 
        self.ax.text(0.5*(self.thetalim[1]-self.thetalim[0]), (1.0+0.03*(self.thetalim[1]-self.thetalim[0])**2)*self.rlim[1],"Correlation", rotation=0.5*(self.thetalim[1]-self.thetalim[0])*180.0/np.pi-90.0)
        rs, ts = np.meshgrid(np.linspace(*self.rlim), np.linspace(*self.thetalim))
        rms = np.sqrt(self.model_std["reference"]**2 + rs**2 - 2.0*self.model_std["reference"]*rs*np.cos(ts))
        self.ax.contourf(ts, rs, rms, 9, alpha=0.3, cmap="RdYlGn_r")
        contours = self.ax.contour(ts, rs, rms, 9, linestyles="--", linewidths=1, alpha=0.5, colors="grey")
        if self.model_std["reference"] > 100.0 or self.model_std["reference"] < 0.1:
            fmt = '%3.2e'
        else:
            fmt = '%3.2f'
        self.ax.clabel(contours, inline=False, fmt=fmt, colors="black")
        t = np.linspace(*self.thetalim)
        r = np.zeros_like(t) + self.model_std["reference"]
        self.ax.plot(t, r, 'k--', label='_')
        xticks = [1.0, 0.99, 0.95, 0.9, 0.8, 0.7, 0.6, 0.4, 0.2, 0.0]
        if self.thetalim[1] > 0.5*np.pi:
            for x in [-0.2, -0.4, -0.6, -0.8, -0.9, -0.95, -0.99 -1.0]:
                if np.arccos(x) > self.thetalim[1]:
                    break
                xticks += [x]
        self.ax.set_xticks(np.arccos(xticks))
        self.ax.set_xticklabels(xticks)

In [ ]:
#| export
class EnhancedMSELoss(nn.Module):
    def __init__(self, alpha=1.5):
        """
        Initialize the enhanced MSE loss module.

        Args:
            alpha (float): Exponential factor to increase penalty for larger errors.
        """
        super(EnhancedMSELoss, self).__init__()
        self.alpha = alpha

    def forward(self, predictions, targets):
        """
        Calculate the enhanced MSE loss.

        Args:
            predictions (torch.Tensor): The predicted values.
            targets (torch.Tensor): The ground truth values.

        Returns:
            torch.Tensor: The calculated loss.
        """
        error = predictions - targets
        mse_loss = torch.mean(error**2)
        enhanced_error = torch.mean(torch.abs(error) ** self.alpha)
        enhanced_mse_loss = mse_loss + enhanced_error
        return enhanced_mse_loss

In [ ]:
#| export
class EnhancedMSEMetric(Metric):
    def __init__(self, alpha=1.5, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.alpha = alpha
        self.add_state("sum_enhanced_error", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, predictions: torch.Tensor, targets: torch.Tensor):
        error = predictions - targets
        mse_loss = torch.mean(error ** 2)
        enhanced_error = torch.mean(torch.abs(error) ** self.alpha)

        self.sum_enhanced_error += (mse_loss + enhanced_error) * targets.numel()
        self.total += targets.numel()

    def compute(self):
        return self.sum_enhanced_error / self.total

In [ ]:
#| export
class PredictionPlottingCallback(L.Callback):
    def __init__(self, num_samples_to_plot=10, rolling_window_size=120):
        """
        Args:
            testDataSet (Dataset): DataLoader for the prediction dataset.
            num_samples_to_plot (int): Number of samples to plot.
            dimensions_to_plot (tuple): Dimensions to plot in the time series.
        """
        super().__init__()
        #self.predictionDataLoader = DataLoader(testDataSet, batch_size=32, shuffle=False, drop_last=True)
        self.num_samples_to_plot = num_samples_to_plot
        self.rolling_window_size = rolling_window_size

    def on_train_epoch_end(self, trainer, pl_module):
        test_dataloader = DataLoader(trainer.datamodule.test, batch_size=32, shuffle=False, drop_last=True)
        try:
            pl_module.eval()
            self.all_predictions = []
            self.all_labels = []

            with torch.no_grad():
                for batch in test_dataloader:
                    features, true_labels = batch
                    # Move data to the same device as the model
                    features, true_labels = features.to(pl_module.device), true_labels.to(pl_module.device)
                    predictions = pl_module(features)

                    # Move predictions and labels to CPU for concatenation
                    self.all_predictions.append(predictions.cpu())
                    self.all_labels.append(true_labels.cpu())

                # Ensure all tensors are on CPU before concatenation
                self.all_predictions = torch.cat(self.all_predictions, dim=0)
                self.all_labels = torch.cat(self.all_labels, dim=0)

                self.plot_time_series(self.all_labels, self.all_predictions, trainer.current_epoch, trainer.datamodule.runoffDataStats)
            pl_module.train()
        except Exception as e:
            print(f"Exception in callback: {e}")

    def rolling_mean(self, data, window_size):
            """Apply a rolling mean to a 2D tensor along the time dimension."""
            cumsum_vec = np.cumsum(np.insert(data, 0, 0, axis=0), axis=0)
            return (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size

    def plot_time_series(self, true_labels, predictions, epoch, runoffStats):
        num_plots = min(len(predictions), self.num_samples_to_plot)
        # Calculate the number of rows and columns for the subplot
        num_cols = int(math.ceil(math.sqrt(num_plots)))
        num_rows = int(math.ceil(num_plots / num_cols))

        true_labels = true_labels*runoffStats[1].data + runoffStats[0].data
        predictions = predictions*runoffStats[1].data + runoffStats[0].data

        # Apply rolling mean
        smoothed_true_labels = self.rolling_mean(true_labels.numpy(), self.rolling_window_size)
        smoothed_predictions = self.rolling_mean(predictions.numpy(), self.rolling_window_size)

        avg_labels = true_labels.mean(dim=0)
        _, top_river_indices = torch.topk(avg_labels, self.num_samples_to_plot, largest=True)    

        plt.figure(figsize=(num_cols * 4, num_rows * 4))  # Adjust the size dynamically based on the number of subplots
        for i, river_index in enumerate(top_river_indices):
            plt.subplot(num_rows, num_cols, i + 1)
            plt.plot(smoothed_predictions[:, river_index], label=f"Smoothed Pred for river {i}")
            plt.plot(smoothed_true_labels[:, river_index], label=f"Smoothed True for river {i}", alpha=0.5)
            plt.legend()

        plt.suptitle(f"Epoch {epoch} Predictions {self.num_samples_to_plot} largest rivers")  
        plt.xlabel("Time")  
        plt.ylabel("Value")  
        plt.savefig(f'figures/time_series_predictions_epoch_{epoch}.png')
        plt.close() 